In [ ]:
import os
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession \
    .builder \
    .config("spark.jars", "/opt/spark/jars/aws-java-sdk-bundle-1.12.505.jar,/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/hadoop-common-3.3.4.jar") \
    .config('spark.master', 'spark://192.168.0.11:7077') \
    .config("spark.sql.warehouse.dir", "spark-warehouse") \
    .enableHiveSupport() \
    .config("spark.cores.max", 2) \
    .config("spark.executor.cores", 2) \
    .config("spark.executor.memory", "2g") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://192.168.0.40:9000") \
    .config("spark.hadoop.fs.s3a.access.key", os.environ['AWS_ACCESS_KEY_ID']) \
    .config("spark.hadoop.fs.s3a.secret.key", os.environ['AWS_SECRET_ACCESS_KEY']) \
    .getOrCreate()

In [ ]:
spark.sql("""
    create table tb_01 (
        id varchar(36),
        code integer,
        option char(8),
        description string,
        value double,
        rate double,
        created_at timestamp,
        updated_at timestamp,
        status boolean
    ) partitioned by (date date) stored as parquet location 's3a://ddfs-bucket-01/tb_01';
""")

In [ ]:
'''
spark.sql("""
    --show databases;
    show tables;
""").show(truncate=False)
'''

'''
spark.sql("""
    --describe database default;
    describe table default.tb_01
""").show(truncate=False)
'''

# add all partition
spark.sql("""
    repair table tb_01;
""")

'''
# add partition
spark.sql("""
    alter table tb_01 add if not exists partition(date = '2022-05-21') partition(date = '2022-11-14');
""")
'''

spark.sql("""
    select count(*) from tb_01
""").show(truncate=False)

'''
spark.sql("""
    show partitions tb_01;
""").show(truncate=False)
'''

In [ ]:
spark.stop()